<a href="https://colab.research.google.com/github/tracydo99/bus118/blob/main/Prompt_Engineering_ReACT_Code_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet openai

import os, re, textwrap, io, sys, traceback
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = "#####"

client = OpenAI()
MODEL = "gpt-4o-mini"

In [ ]:
def react_solve(task_prompt: str, max_rounds=3):
    """Reason + Act + Reflect cycle for code generation."""
    system = """You are a careful Python engineer.
You always reason first (Plan) then produce a single, executable code block in ```python``` fences."""
    user = f"""
TASK:
{task_prompt}

FORMAT:
1. "Plan:" (2–5 bullet points explaining logic)
2. One ```python``` code block that defines the functions, runs tests, and prints results.
"""

    for round_idx in range(1, max_rounds + 1):
        reply = chat(system, user)
        code = extract_python_block(reply)

        if not code:
            user = "Please output a valid Python code block only."
            continue

        result = run_user_code(code)

        # ✅ Fixed — no reference to stderr
        if result["ok"]:
            print(f"✅ Success on round {round_idx}")
            print("---- OUTPUT ----\n", result["stdout"])
            print("---- CODE ----\n", textwrap.shorten(code, width=800))
            return
        else:
            print(f"❌ Error round {round_idx}: {result['error']}")
            user = f"Fix this error and reprint working code:\n{result['error']}"

In [ ]:
task = """
Write a Python program that converts between Centigrade and Fahrenheit.
- Formula 1: F = 32 + C * (180.0 / 100.0)
- Formula 2: C = (F - 32) * (100.0 / 180.0)
- The program should define c_to_f() and f_to_c().
- Print at least three test cases (include one negative temperature).
- No user input; just print sample results clearly labeled.
"""
react_solve(task)

✅ Success on round 1
---- OUTPUT ----
 Centigrade to Fahrenheit:
0°C = 32.00°F
100°C = 212.00°F
-40°C = -40.00°F

Fahrenheit to Centigrade:
32°F = 0.00°C
212°F = 100.00°C
-40°F = -40.00°C

---- CODE ----
 def c_to_f(celsius): """Convert Centigrade to Fahrenheit.""" return 32 + celsius * (180.0 / 100.0) def f_to_c(fahrenheit): """Convert Fahrenheit to Centigrade.""" return (fahrenheit - 32) * (100.0 / 180.0) # Test cases test_cases_c_to_f = [0, 100, -40] # 0°C, 100°C, -40°C test_cases_f_to_c = [32, 212, -40] # 32°F, 212°F, -40°F # Print results for Centigrade to Fahrenheit print("Centigrade to Fahrenheit:") for c in test_cases_c_to_f: f = c_to_f(c) print(f"{c}°C = {f:.2f}°F") # Print results for Fahrenheit to Centigrade print("\nFahrenheit to Centigrade:") for f in test_cases_f_to_c: c = f_to_c(f) print(f"{f}°F = {c:.2f}°C")


In [ ]:
def c_to_f(celsius):
    """Convert Centigrade to Fahrenheit."""
    return 32 + celsius * (180.0 / 100.0)

def f_to_c(fahrenheit):
    """Convert Fahrenheit to Centigrade."""
    return (fahrenheit - 32) * (100.0 / 180.0)

# Test cases
test_cases_c_to_f = [0, 100, -40]   # 0°C, 100°C, -40°C
test_cases_f_to_c = [32, 212, -40]  # 32°F, 212°F, -40°F

# Print results for Centigrade to Fahrenheit
print("Centigrade to Fahrenheit:")
for c in test_cases_c_to_f:
    f = c_to_f(c)
    print(f"{c}°C = {f:.2f}°F")

# Print results for Fahrenheit to Centigrade
print("\nFahrenheit to Centigrade:")
for f in test_cases_f_to_c:
    c = f_to_c(f)
    print(f"{f}°F = {c:.2f}°C")

Centigrade to Fahrenheit:
0°C = 32.00°F
100°C = 212.00°F
-40°C = -40.00°F

Fahrenheit to Centigrade:
32°F = 0.00°C
212°F = 100.00°C
-40°F = -40.00°C
